# 🔓 RSA Attack Level 1 – Kelompok 4

Notebook ini menunjukkan bagaimana RSA bisa diserang jika modulus `n` terlalu kecil.

Langkah serangan:
1. Faktorkan `n` menjadi `p` dan `q`
2. Hitung `phi(n)`
3. Hitung `d = mod_inverse(e, phi)`
4. Dekripsi ciphertext dengan `m = c^d mod n`

Asumsi: penyerang hanya tahu `e`, `n`, dan `ciphertext`


In [1]:
# Import library yang diperlukan
from sympy import isprime, mod_inverse, gcd
from math import isqrt
import random

# Fungsi untuk menghasilkan bilangan prima acak
def generate_prime(bits=8):
    while True:
        num = random.randint(2**(bits - 1), 2**bits - 1)  # Angka acak dalam bit tertentu
        if isprime(num):
            return num

# Fungsi faktorisasi n menjadi p dan q (brute force untuk n kecil)
def factor_n(n):
    for i in range(2, isqrt(n) + 1):
        if n % i == 0:
            p = i
            q = n // i
            if isprime(p) and isprime(q):
                return p, q
    return None, None

# ===== Generate p dan q secara acak =====
p = generate_prime()
q = generate_prime()
while q == p:  # Pastikan p dan q berbeda
    q = generate_prime()

n = p * q                     # Modulus n
phi = (p - 1) * (q - 1)       # Euler's totient

# ===== Generate e secara acak, pastikan gcd(e, phi) == 1 =====
while True:
    e = random.randrange(3, phi, 2)  # Pilih e ganjil
    if gcd(e, phi) == 1:
        break

# ===== Hitung kunci privat d =====
d = mod_inverse(e, phi)

# Tampilkan kunci RSA
print("Bilangan prima p:", p)
print("Bilangan prima q:", q)
print("Modulus n:", n)
print("phi(n):", phi)
print("\nKunci publik (e, n):", (e, n))
print("Kunci privat (d, n):", (d, n))

# ===== Simulasi korban: enkripsi pesan =====
plaintext = "Attack Level 1 RSA Daffa Dimas Nova"
ascii_plain = [ord(c) for c in plaintext]
ciphertext = [pow(m, e, n) for m in ascii_plain]
print("\nPlaintext:", plaintext)
print("ASCII plaintext:", ascii_plain)
print("Ciphertext:", ciphertext)

# ===== Simulasi penyerang: faktorisasi n dan dekripsi =====
p_, q_ = factor_n(n)
if not p_ or not q_:
    raise ValueError("Gagal faktorisasi n. Mungkin n terlalu besar untuk brute force.")

phi_ = (p_ - 1) * (q_ - 1)
d_ = mod_inverse(e, phi_)
decrypted_ascii = [pow(c, d_, n) for c in ciphertext]

try:
    decrypted_text = ''.join(chr(c) for c in decrypted_ascii)
except ValueError:
    decrypted_text = "[Beberapa karakter ASCII tidak valid]"

Bilangan prima p: 149
Bilangan prima q: 211
Modulus n: 31439
phi(n): 31080

Kunci publik (e, n): (3013, 31439)
Kunci privat (d, n): (18877, 31439)

Plaintext: Attack Level 1 RSA Daffa Dimas Nova
ASCII plaintext: [65, 116, 116, 97, 99, 107, 32, 76, 101, 118, 101, 108, 32, 49, 32, 82, 83, 65, 32, 68, 97, 102, 102, 97, 32, 68, 105, 109, 97, 115, 32, 78, 111, 118, 97]
Ciphertext: [12774, 31403, 31403, 3052, 6969, 7573, 8478, 4738, 23390, 14209, 23390, 29073, 8478, 6561, 8478, 27033, 15326, 12774, 8478, 3629, 3052, 7052, 7052, 3052, 8478, 3629, 20518, 4975, 3052, 3996, 8478, 22126, 10822, 14209, 3052]


In [2]:
# Tampilkan hasil dari proses serangan
print("Faktorisasi n → p:", p, "q:", q)
print("phi(n):", phi)
print("d (private key):", d)
print("\nHasil dekripsi (ASCII):", decrypted_ascii)
print("Plaintext:", decrypted_text)


Faktorisasi n → p: 149 q: 211
phi(n): 31080
d (private key): 18877

Hasil dekripsi (ASCII): [65, 116, 116, 97, 99, 107, 32, 76, 101, 118, 101, 108, 32, 49, 32, 82, 83, 65, 32, 68, 97, 102, 102, 97, 32, 68, 105, 109, 97, 115, 32, 78, 111, 118, 97]
Plaintext: Attack Level 1 RSA Daffa Dimas Nova


## ✅ Kesimpulan

RSA Attack Level 1 berhasil dilakukan karena ukuran `n` kecil, sehingga faktorisasi bisa dilakukan secara langsung.

**Rekomendasi keamanan:**
- Gunakan modulus `n` minimal 2048-bit
- Hindari `e`, `n` kecil untuk penggunaan nyata

RSA hanya aman jika parameter dibuat dengan benar.
